# Check Google Search Volume Indices by category

In [1]:
from pathlib import Path

import pandas as pd
import plotly.graph_objects as go
from dateutil.relativedelta import relativedelta

In [2]:
# Google SVI
data_dir = Path("../../data/raw/google_category/monthly")

dfs = []
for file in list(data_dir.glob("*.csv")):
    _df = pd.read_csv(file, index_col=0)
    _df.index = pd.to_datetime(_df.index).map(lambda x: x + relativedelta(months=1, days=-1))
    dfs.append(_df)
df_svi = pd.concat(dfs, axis=1)

# UNRATE
file = Path("../../data/raw/macro/UNRATE.csv")
df_unrate = pd.read_csv(file, index_col=0)
df_unrate.index = pd.to_datetime(df_unrate.index).map(lambda x: x + relativedelta(months=1, days=-1))

# IC
file = Path("../../data/raw/macro/ICSA.csv")
df_icsa = pd.read_csv(file, index_col=0)
df_icsa.index = pd.to_datetime(df_icsa.index)

In [3]:
df = pd.concat([df_unrate, df_svi], axis=1).dropna()

traces = []
for col in df.columns:
    trace = go.Scatter(
        x = df.index,
        y = df.loc[:, col],
        name=col,
    )
    traces.append(trace)
traces.append(
    go.Scatter(
        x = df_icsa.index,
        y = df_icsa.iloc[:, 0] / df_icsa.iloc[:, 0].max() * 100,
        name="IC"
    )
)
layout = go.Layout(
    width=1400,
    height=800,
    font_size=15,
    legend={"x":0.99, "y":1.1, "xanchor":"right", "yanchor":"bottom","orientation":"h"},
    xaxis={"title":"Date"},
    yaxis={"title":"SVI", "side":"left"},
    hovermode="x",
    template="plotly_dark",
)
fig = go.Figure(traces, layout=layout)
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            {
                "count":i, "label":f"last {i} year", "step":"year",
            } for i in [1, 3, 5, 10]
        ],
        bgcolor="#2F301F"
    )
)
fig.show()

Welfare & Unemployment (706) well works. The others don't fit to unemployment rate.